#### Question 1

Posted on absalon is a press release from the European Securities and Markets Authoriy regarding its decision to prohibit sales of binary options to retail investors. Read the [article](https://www.esma.europa.eu/press-news/esma-news/esma-agrees-prohibit-binary-options-and-restrict-cfds-protect-retail-investors) and answer the following questions:

1. What is the motivation behind this decision? How does this relate to the models we have seen in class?

2. What eﬀect will this decision have on liquidity in the binary options market?

3. Measures announced in the press release diﬀer between the binary option and CFD markets. How will the eﬀects of the regulation be diﬀerent across the two markets?

Be concise and to the point. Please try to keep your answer less than 100 words (but not just one sentense).

1. The motivation behind the decision is to protect retail investors from significant investor protection concerns related to the complexity and lack of transparency of binary options and CFDs, as well as excessive leverage, structural expected negative return, and issues related to their marketing and distribution. This relates to the models we have seen in class that highlight the importance of transparency and appropriate risk management in financial markets to protect investors.

2. The decision to prohibit sales of binary options to retail investors is expected to reduce liquidity in the binary options market as retail investors will no longer be able to trade these instruments. However, this is seen as a necessary measure to protect retail investors from the risks associated with binary options, particularly their structural expected negative return and embedded conflict of interest between providers and their clients.

3. The measures announced in the press release will have different effects on the binary option and CFD markets. For binary options, the prohibition on the marketing, distribution, or sale of these instruments to retail investors will prevent retail investors from trading these instruments, whereas for CFDs, the restrictions on leverage, margin close-out, negative balance protection, and incentives, as well as the requirement for a standardised risk warning, will limit the risks associated with these instruments for retail investors.

#### Question 2 Trade data 
In the following use the trade data in tqBAC.csv.  Denote trade prices by $p_t$ and mid-quotes by $m_t$.
* Sign each trade based on Lee-ready algorithm 
* Calcualte Spread, Effective Spread and Realized Spread by EXCHANGES
* Examine order correlation 

#### Import the relevant modules

In [8]:
### In this project, I will use datatable (quicker) rather pandas to manipulate data. You can see which one is more intuitive for you 
!pip install datatable
from datetime import datetime
import datatable as dt
from datatable import dt, f, by, update
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
import os 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 MB 9.7 MB/s eta 0:00:00


1. Sign each trade based on Lee-Ready algorithm:

In [9]:
import pandas as pd

# read the trade data into a pandas dataframe
df = pd.read_csv('tqBAC.csv')

# initialize the trade sign column
df['TradeSign'] = 0

# loop over the trades and assign trade signs
for i in range(1, len(df)):
    pt = df.loc[i, 'PRICE']
    mt = (df.loc[i, 'BID'] + df.loc[i, 'OFR']) / 2
    pt_1 = df.loc[i-1, 'PRICE']
    
    if pt > mt:
        df.loc[i, 'TradeSign'] = 1
    elif pt < mt:
        df.loc[i, 'TradeSign'] = -1
    else:
        if pt > pt_1:
            df.loc[i, 'TradeSign'] = 1
        elif pt < pt_1:
            df.loc[i, 'TradeSign'] = -1


2. Calculate spread, effective spread, and realized spread by exchanges:

In [10]:
# calculate spread
df['Spread'] = df['OFR'] - df['BID']

# calculate effective spread
df['EffectiveSpread'] = 2 * df['Spread'] / (df['OFR'] + df['BID'])

# calculate realized spread
df['MidQuote'] = (df['BID'] + df['OFR']) / 2
df['MidQuoteLag'] = df['MidQuote'].shift(1)
df['MidQuoteReturn'] = np.log(df['MidQuote'] / df['MidQuoteLag'])
df['RealizedSpread'] = np.exp(df['MidQuoteReturn'].rolling(600).sum()) - 1
df = df.drop(columns=['MidQuote', 'MidQuoteLag', 'MidQuoteReturn'])
df['RealizedSpread'] = df['RealizedSpread'].fillna(0)

# group by exchanges and calculate mean values
df_grouped = df.groupby('EX').agg({
    'Spread': 'mean',
    'EffectiveSpread': 'mean',
    'RealizedSpread': 'mean'
})


3. Examine order correlation:

In [27]:
##just checking if there's missing data
print(df.isnull().sum())

date                 0
SYMBOL               0
EX                   0
PRICE                0
SIZE                 0
COND                 0
BID                  0
BIDSIZ               0
OFR                  0
OFRSIZ               0
TradeSign            0
Spread               0
EffectiveSpread      0
RealizedSpread       0
OrderCorr          600
dtype: int64


In [29]:
df

,date,SYMBOL,EX,PRICE,SIZE,COND,BID,BIDSIZ,OFR,OFRSIZ,TradeSign,Spread,EffectiveSpread,RealizedSpread,OrderCorr
0,2017-09-19T14:30:00.009Z,BAC,P,24.700,20000,,24.72,6,24.74,100,0,0.02,0.000809,0.0,NaN
1,2017-09-19T14:30:01.361Z,BAC,T,24.710,300,,24.72,6,24.74,100,-1,0.02,0.000809,0.0,NaN
2,2017-09-19T14:30:01.525Z,BAC,P,24.700,100,,24.72,6,24.74,100,-1,0.02,0.000809,0.0,NaN
3,2017-09-19T14:30:02.990Z,BAC,N,24.730,429950,,24.72,6,24.74,100,1,0.02,0.000809,0.0,NaN
4,2017-09-19T14:30:02.996Z,BAC,N,24.720,100,,24.72,35,24.74,420,-1,0.02,0.000809,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24455,2017-09-19T20:59:59.993Z,BAC,B,24.870,137,,24.86,3020,24.87,2071,1,0.01,0.000402,0.0,0.249432
24456,2017-09-19T21:00:00.009Z,BAC,P,24.870,28400,,24.86,3020,24.87,1956,1,0.01,0.000402,0.0,0.251705
24457,2017-09-19T21:00:00.013Z,BAC,A,24.860,100,,24.86,3020,24.87,1956,-1,0.01,0.000402,0.0,0.248775
24458,2017-09-19T21:00:00.213Z,BAC,P,24.870,28400,,24.86,3020,24.87,1956,1,0.01,0.000402,0.0,0.245857


In [30]:
order_sign = np.sign(df['SIZE'])
order_sign_autocorr = np.corrcoef(order_sign[:-1], order_sign[1:])
print(order_sign_autocorr[0,1])

nan


/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]


4.  Calculate Mean Spreads at the Exchange Level

In [36]:
# Calculate mid-price
df['mid_price'] = (df['BID'] + df['OFR']) / 2

# Calculate spread
df['spread'] = df['OFR'] - df['BID']

# Calculate mean spread at the exchange level
mean_spread_by_exchange = df.groupby(['date', 'EX'])['spread'].mean()
print(mean_spread_by_exchange)



date                      EX
2017-09-19T14:30:00.009Z  P     0.02
2017-09-19T14:30:01.361Z  T     0.02
2017-09-19T14:30:01.525Z  P     0.02
2017-09-19T14:30:02.990Z  N     0.02
2017-09-19T14:30:02.996Z  N     0.02
                                ... 
2017-09-19T20:59:59.993Z  B     0.01
2017-09-19T21:00:00.009Z  P     0.01
2017-09-19T21:00:00.013Z  A     0.01
2017-09-19T21:00:00.213Z  P     0.01
2017-09-19T21:00:00.344Z  T     0.01
Name: spread, Length: 24460, dtype: float64


#### Lee and Ready algorithm 
A typical classification is 
 *   buyer-initiated  if pt > mt
 *   buyer-initiated  if pt = mt and pt < pt−1 (downtick)
 *   seller-initiated if pt < mt
 *   seller-initiated if pt = mt and pt > pt−1 (uptick)

#### Calcualte Spread, Effective Spread and Realized Spread by EXCHANGES
Note that for realized spread, using mid-quote in 10 mins 
1. calculate correlation of three spreads
2. plot time series of three spreads by hour 
2. calculate mean spreads at the Exchange level 

#### Order Sign Correlation
1. autocorrelation plot of order sign 
2. re-produce the above figure in log term (both x-axis and y-axis are in log term)

#### Question 3 Quote data
Data BAC_nbbo.csv is order bood data (only the best quotes), with each row one of the price or size at the best bid or ask changes which corresponds to change in the supply or demand. 
* Calculate order imbalance OFI (keep only Nasdaq exchanges)
* Aggregate OFI to second level (take summation)

##### Order Imbalance
Order flow imbalance represents the changes in supply and demand. 
* Best bid or size at the best bid increase -> increase in demand.
* Best bid or size at the best bid decreases -> decrease in demand.
* Best ask decreases or size at the best ask increases -> increase in supply.
* Best ask increases or size at the best ask decreases -> decrease in supply.

Mathematically we summarise these four effects at from time $n-1$ to $n$ as:
$$e_n = I_{B_n \geq B_{n-1}} q_n - I_{B_n \leq B_{n-1}} q_{n-1} - I_{A_n \leq A_{n-1}} q_n + I_{A_n \geq A_{n-1}} q_{n-1} $$
where $B_n$ is the beset Bid price at time $n$ and $q_n$ is the size at those prices 

In [42]:
import pandas as pd

data = pd.read_csv("BAC_nbbo.csv")


In [43]:
# Creat second stamp
data['TIME_M'] = pd.to_datetime(data['DATE'].astype(str) + ' ' + data['TIME_M'])
data.set_index('TIME_M', inplace=True)
data_sec = data.resample('1S').last()

In [44]:
## only keep trading hours
trading_hours = [('14:30', '21:00')]
trading_hours = [(pd.to_datetime(start, format='%H:%M') - pd.Timedelta(hours=5),
                  pd.to_datetime(end, format='%H:%M') - pd.Timedelta(hours=5) + pd.Timedelta(days=1))
                 for start, end in trading_hours]
mask = pd.Series(False, index=data_sec.index)
for start, end in trading_hours:
    mask = mask | ((data_sec.index >= start) & (data_sec.index < end))
data_sec = data_sec.loc[mask]


In [46]:
# notice the extreme values in BID and ASK!! 
# need to clean data 
# first, remove negative spreads
# then outlier quotes
#Calculate the bid-ask spread for each row by subtracting the best ask price from the best bid price.
data_sec['spread'] = data_sec['ASK'] - data_sec['BID']
#Remove rows with negative spreads using a boolean mask.
data_sec = data_sec[data_sec['spread'] >= 0]
#Calculate the z-score of the bid and ask prices using the median and median absolute deviation (MAD) as robust measures of location and scale, respectively.
from statsmodels.robust.scale import mad

bid_med = data_sec['BID'].median()
ask_med = data_sec['ASK'].median()
bid_mad = mad(data_sec['BID'])
ask_mad = mad(data_sec['ASK'])
data_sec['bid_z'] = (data_sec['BID'] - bid_med) / bid_mad
data_sec['ask_z'] = (data_sec['ASK'] - ask_med) / ask_mad
#Define a threshold for outlier quotes as a number of MADs from the median, and remove rows with bid or ask z-scores greater than the threshold.
outlier_threshold = 5
data_sec = data_sec[(data_sec['bid_z'].abs() <= outlier_threshold) &
                    (data_sec['ask_z'].abs() <= outlier_threshold)]


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [62]:
print(data_sec.columns)

Index(['DATE', 'EX', 'BID', 'BIDSIZ', 'ASK', 'ASKSIZ', 'QU_COND', 'QU_SEQNUM',
       'NATBBO_IND', 'QU_CANCEL', 'QU_SOURCE', 'SYM_ROOT', 'SYM_SUFFIX',
       'spread', 'bid_z', 'ask_z'],
      dtype='object')


In [70]:
data_sec['TIME_M'] = data_sec.index.time


In [73]:
# Aggregate by second 
# Construct return as log difference of last mid price and first mid price of each second 
data_sec['datetime'] = pd.to_datetime(data_sec['DATE'].astype(str) + ' ' + data_sec['TIME_M'])
data_sec.set_index('datetime', inplace=True)
data_sec = data_sec.resample('1S').mean()
data_sec['mid'] = (data_sec['BID'] + data_sec['ASK']) / 2
data_sec['mid_return'] = np.log(data_sec['mid']).diff()

Using OFI to generate trading signal: first do train/test split by selecting the first 70% of the data




In [78]:
print(data_sec.columns)

Index(['EX', 'BID', 'BIDSIZ', 'ASK', 'ASKSIZ', 'QU_COND', 'QU_SEQNUM',
       'NATBBO_IND', 'QU_CANCEL', 'QU_SOURCE', 'SYM_ROOT', 'SYM_SUFFIX',
       'spread', 'bid_z', 'ask_z', 'TIME_M', 'mid', 'mid_return'],
      dtype='object')


In [80]:
ofi = (data_sec['BID'] >= data_sec['BID'].shift()) * data_sec['BIDSIZ'] \
    - (data_sec['BID'] <= data_sec['BID'].shift()) * data_sec['BIDSIZ'].shift() \
    - (data_sec['ASK'] <= data_sec['ASK'].shift()) * data_sec['ASKSIZ'] \
    + (data_sec['ASK'] >= data_sec['ASK'].shift()) * data_sec['ASKSIZ'].shift()

ofi = ofi.loc['2015-01-02':'2022-01-01']  # Only keep Nasdaq exchanges and trading hours

In [82]:
train_size = int(len(ofi) * 0.7)
train_ofi = ofi.iloc[:train_size]
test_ofi = ofi.iloc[train_size:]

In [90]:
# explainatry power for test sample 
# Calculate log returns for test period
mid_price_test = (data_sec['BID'].loc[test_ofi.index] + data_sec['ASK'].loc[test_ofi.index]) / 2
log_return_test = np.log(mid_price_test).diff()

# Remove missing values from OFI and log return data
test_ofi_clean = test_ofi.dropna()
log_return_test_clean = log_return_test.loc[test_ofi_clean.index]

# Calculate correlation between OFI and log returns
corr_test = test_ofi_clean.corr(log_return_test_clean)

print(f"Correlation between OFI and log returns for test period: {corr_test:.2f}")


Correlation between OFI and log returns for test period: nan


##### Construct a Predictive Trading Signal
BUT! The above analysis is in-sample. We want to see out-sample results. 

In [98]:
def OFI(data):
  demand_change = data['BIDSIZ'].diff()
  supply_change = data['ASKSIZ'].diff()

    # Calculate OFI
  OFI = demand_change - supply_change

    # Only keep data from Nasdaq exchanges
  OFI_NASDAQ = OFI.loc[data['EX'] == 'Q']

  return OFI_NASDAQ

In [100]:
from sklearn.linear_model import LinearRegression

In [115]:
data_sec['midprice'] = (data_sec['BID'] + data_sec['ASK']) / 2


In [118]:
# Test whether OFI can explain FUTURE return variations
# Split sample to test and train samples again 
# Test whether lagged OFI can predict FUTURE return
# Split data into train and test samples
n_obs = data_sec.shape[0]
n_train = int(n_obs * 0.7)

train_data = data_sec.iloc[:n_train, :]
test_data = data_sec.iloc[n_train:, :]
test_data['ret_1'] = np.log(test_data['midprice'].shift(-1)) - np.log(test_data['midprice'])


In [122]:
def calc_ofi(data):
    ofi = []
    for i in range(1, len(data)):
        # check for changes in the best bid and ask prices and sizes
        bid_change = data['BID'].iloc[i] != data['BID'].iloc[i-1] or data['BIDSIZ'].iloc[i] != data['BIDSIZ'].iloc[i-1]
        ask_change = data['ASK'].iloc[i] != data['ASK'].iloc[i-1] or data['ASKSIZ'].iloc[i] != data['ASKSIZ'].iloc[i-1]
        
        if bid_change and not ask_change:
            # increase in demand
            ofi.append(data['BIDSIZ'].iloc[i] - data['BIDSIZ'].iloc[i-1])
        elif ask_change and not bid_change:
            # increase in supply
            ofi.append(-(data['ASKSIZ'].iloc[i] - data['ASKSIZ'].iloc[i-1]))
        else:
            # no change or simultaneous changes
            ofi.append(0)
    
    return ofi


In [124]:
train_size = int(len(data) * 0.7)
train = data.iloc[:train_size]


In [ ]:
train['ofi'] = calc_ofi(data_sec[data_sec.index.isin(train.index)])

In [ ]:
from sklearn.linear_model import LinearRegression

# Create lagged OFI variable
train_data['ofi_lag'] = train_data['ofi'].shift(1)

# Fit linear regression model
lr = LinearRegression()
lr.fit(train_data[['ofi_lag']], train_data['ret_1'])

# Predict test set returns using fitted model
test_data['ret_1_pred'] = lr.predict(test_data[['ofi']])

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(test_data['ret_1'], test_data['ret_1_pred'])
print(f"R-squared: {r2}")

In [114]:
data_sec.columns

Index(['EX', 'BID', 'BIDSIZ', 'ASK', 'ASKSIZ', 'QU_COND', 'QU_SEQNUM',
       'NATBBO_IND', 'QU_CANCEL', 'QU_SOURCE', 'SYM_ROOT', 'SYM_SUFFIX',
       'spread', 'bid_z', 'ask_z', 'TIME_M', 'mid', 'mid_return'],
      dtype='object')

In [ ]:
# explainatry power for test sample
# Calculate OFI for test data
test_ofi = calc_ofi(test)

# Calculate one-second ahead returns for test data
test['ret'] = np.log(test['midprice'].shift(-1)) - np.log(test['midprice'])

# Calculate the mean return for each OFI value
test_ofi_ret = test.groupby('ofi')['ret'].mean()

# Calculate the explanatory power of OFI on test returns
test_r2 = np.corrcoef(test['ofi'].values, test['ret'].values)[0, 1] ** 2

# Print the test R-squared value
print("Test R-squared: ", test_r2)


In [ ]:
# plots cummulative return of the strategy using signal from past OFI 
# create trading signal using lagged OFI
test['signal'] = np.where(test['lag_ofi'] > 0, 1, -1)

# calculate one-second ahead return using midprice
test['return'] = np.log(test['midprice'].shift(-1)) - np.log(test['midprice'])

# calculate strategy return
test['strategy_return'] = test['signal'] * test['return']

# calculate cumulative strategy return
test['cum_strategy_return'] = test['strategy_return'].cumsum()

# plot cumulative strategy return
plt.plot(test.index, test['cum_strategy_return'])
plt.title('Cumulative Return of Trading Strategy')
plt.xlabel('Time')
plt.ylabel('Cumulative Return')
plt.show()
